# 2022 Championship EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [2]:
team_stats = pd.read_csv('../footy_data/championship_teams_2022.csv')
games = pd.read_csv('../footy_data/championship_games_2022.csv')

In [3]:
games = games[0:36]
games
#Dropping playoff matches

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1628275500,Aug 06 2021 - 6:45pm,complete,NaN,AFC Bournemouth,West Bromwich Albion,Dean Whitestone,1,0.0,0.0,...,2.30,3.40,3.10,1.33,1.85,3.15,6.00,1.80,2.05,Vitality Stadium (Bournemouth- Dorset)
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Blackburn Rovers,Swansea City,David Webb,1,0.0,0.0,...,2.20,3.25,3.60,1.38,2.15,3.65,7.25,1.95,1.87,Ewood Park (Blackburn- Lancashire)
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Bristol City,Blackpool,Andy Davies,1,0.0,0.0,...,2.20,3.35,3.45,1.39,2.10,3.45,6.75,1.95,1.91,Ashton Gate (Bristol)
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Cardiff City,Barnsley,Steve Martin,1,0.0,0.0,...,2.50,3.20,3.00,1.45,2.25,3.85,7.50,1.95,1.91,Cardiff City Stadium (Cardiff (Caerdydd))
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Derby County,Huddersfield Town,Geoff Eltringham,1,0.0,0.0,...,2.60,3.35,2.80,1.43,2.15,3.70,7.00,1.95,1.87,Pride Park Stadium (Derby)
5,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Luton Town,Peterborough United,Andy Woolmer,1,0.0,0.0,...,2.60,3.35,2.20,1.30,2.00,3.25,6.00,1.73,2.00,Kenilworth Road (Luton- Bedfordshire)
6,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Preston North End,Hull City,Gavin Ward,1,0.0,0.0,...,2.35,3.30,3.20,1.39,2.10,3.45,6.50,1.91,1.95,Deepdale (Preston)
7,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Queens Park Rangers,Millwall,Oliver Langford,1,0.0,0.0,...,2.15,3.35,3.65,1.42,2.10,3.55,6.75,1.91,1.91,Kiyan Prince Foundation Stadium (London)
8,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Stoke City,Reading,Matt Donohue,1,0.0,0.0,...,2.10,3.25,3.90,1.53,2.45,4.35,8.75,2.20,1.69,bet365 Stadium (Stoke-on-Trent- Staffordshire)
9,1628362800,Aug 07 2021 - 7:00pm,complete,NaN,Sheffield United,Birmingham City,Tim Robinson,1,0.0,0.0,...,1.80,3.40,5.25,1.49,2.40,4.35,8.75,2.25,1.67,Bramall Lane (Sheffield)


In [4]:
xg_view = games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]
xg_view

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
0,AFC Bournemouth,West Bromwich Albion,1.08,2.03,2,2
1,Blackburn Rovers,Swansea City,2.29,1.15,2,1
2,Bristol City,Blackpool,1.28,0.93,1,1
3,Cardiff City,Barnsley,1.52,1.23,1,1
4,Derby County,Huddersfield Town,1.27,1.85,1,1
5,Luton Town,Peterborough United,1.73,0.78,3,0
6,Preston North End,Hull City,1.01,1.87,1,4
7,Queens Park Rangers,Millwall,1.52,1.58,1,1
8,Stoke City,Reading,2.02,1.60,3,2
9,Sheffield United,Birmingham City,1.43,1.13,0,1


# Goal Analysis

In [5]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100

In [6]:
print(f'There were {season_goal_total} goals scored this season')

There were 99 goals scored this season


In [7]:
print(f'There were {season_1h_goal_total} goals scored in the 1H this season, ')

There were 38 goals scored in the 1H this season, 


In [8]:
round(season_1h_goal_total/season_goal_total, 2)
#% of first half goals

0.38

In [9]:
games['total_goals_at_half_time'].count()

36

In [10]:
games['total_goals_at_half_time'].value_counts()

1    16
0    11
2     6
3     2
4     1
Name: total_goals_at_half_time, dtype: int64

In [11]:
(191/557) * 100
#0-0 halftime

34.29084380610413

0.7127468581687613

# Adding more columns

In [13]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [14]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats.head()

,common_name,xg_against_avg_overall
0,Stoke City,0.64
1,West Bromwich Albion,0.41
2,Middlesbrough,0.69
3,AFC Bournemouth,1.06
4,Hull City,0.74


In [15]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [16]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [22]:
fill_stats('Fulham')
fill_stats('Blackburn Rovers')
fill_stats('West Bromwich Albion')
fill_stats('Queens Park Rangers')
fill_stats('AFC Bournemouth')
fill_stats('Middlesbrough')
fill_stats('Barnsley')
fill_stats('Millwall')
fill_stats('Sheffield United')
fill_stats('Reading')
fill_stats('Cardiff City')
fill_stats('Nottingham Forest')
fill_stats('Hull City')
fill_stats('Blackpool')
fill_stats('Luton Town')
fill_stats('Stoke City')
fill_stats('Swansea City')
fill_stats('Derby County')
fill_stats('Huddersfield Town')
fill_stats('Preston North End')
fill_stats('Coventry City')
fill_stats('Peterborough United')
fill_stats('Birmingham City')
fill_stats('Bristol City')

#Reading in stats

# Some more EDA

Let's write a function to help fill xg in our games df

In [23]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go



xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats.head()

xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

fill_stats('Fulham')
fill_stats('Blackburn Rovers')
fill_stats('West Bromwich Albion')
fill_stats('Queens Park Rangers')
fill_stats('AFC Bournemouth')
fill_stats('Middlesbrough')
fill_stats('Barnsley')
fill_stats('Millwall')
fill_stats('Sheffield United')
fill_stats('Reading')
fill_stats('Cardiff City')
fill_stats('Nottingham Forest')
fill_stats('Hull City')
fill_stats('Blackpool')
fill_stats('Luton Town')
fill_stats('Stoke City')
fill_stats('Swansea City')
fill_stats('Derby County')
fill_stats('Huddersfield Town')
fill_stats('Preston North End')
fill_stats('Coventry City')
fill_stats('Peterborough United')
fill_stats('Birmingham City')
fill_stats('Bristol City')

#Reading in stats

# Some more EDA

# Over/Under 2.5?

In [19]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [20]:
games['over/under'].value_counts()

1    20
0    16
Name: over/under, dtype: int64

# xG Over/Under 2.5? 

In [ ]:
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['xg_over/under_1.99'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 1.99, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [ ]:
games['xg_over/under'].value_counts()

In [ ]:
games['xg_over/under_1.99'].value_counts()

In [ ]:
369/552

# First Week

In [ ]:
games['over/under_1.5'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 1.5, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [ ]:
first_matchweek = games[0:12]

In [ ]:
first_matchweek['over/under'].value_counts()

In [ ]:
first_matchweek['over/under_1.5'].value_counts()

In [ ]:
first_matchweek['xg_over/under'].value_counts()

In [ ]:
first_matchweek['xg_over/under_1.99'].value_counts()

# Let's take out the final week now and check percentage

In [ ]:
first_41 = games[0:540]
first_41['over/under'].value_counts()

In [ ]:
217/540
# Percentage of games that went over

# When were these goals scored?

Second half stoppage time goals will start at 91 i.e. 90+1 will be 91
First half stoppage time goals will start at 100 i.e. 45+1 = 110 ----

In [ ]:
goal_timings['All_goal_timings'].unique()

In [ ]:
goal_timings['All_goal_timings'].value_counts()

Change stoppage time goals so we can cast them as floats

In [ ]:
goal_timings = goal_timings.replace(to_replace="90'1", value = "91")
goal_timings = goal_timings.replace(to_replace="90'2", value = "92")
goal_timings = goal_timings.replace(to_replace="90'3", value = "93")
goal_timings = goal_timings.replace(to_replace="90'4", value = "94")
goal_timings = goal_timings.replace(to_replace="90'5", value = "95")
goal_timings = goal_timings.replace(to_replace="90'6", value = "96")
goal_timings = goal_timings.replace(to_replace="90'7", value = "97")
goal_timings = goal_timings.replace(to_replace="90'8", value = "98")
goal_timings = goal_timings.replace(to_replace="90'9", value = "99")
goal_timings = goal_timings.replace(to_replace="90'10", value = "100")
goal_timings = goal_timings.replace(to_replace="90'11", value = "101")
goal_timings = goal_timings.replace(to_replace="90'12", value = "102")
goal_timings = goal_timings.replace(to_replace="45'1", value = "110")
goal_timings = goal_timings.replace(to_replace="45'2", value = "111")
goal_timings = goal_timings.replace(to_replace="45'3", value = "112")
goal_timings = goal_timings.replace(to_replace="45'4", value = "113")

In [ ]:
goal_timings['All_goal_timings'] = goal_timings['All_goal_timings'].astype('int')

In [ ]:
timing_totals = goal_timings['All_goal_timings'].value_counts()
timing_totals = pd.DataFrame(timing_totals)

In [ ]:
timing_totals = timing_totals.rename_axis("minute", axis="columns")
timing_totals = timing_totals.rename(columns={"All_goal_timings":"count"})

In [ ]:
timing_totals['section'] = ""
timing_totals.head()

# In how many games did a team have more xG but lose?